In [1]:
#爬取柏克萊書籍  書名:python  價格, 圖片
import requests # 跟前檯小姐姐拿資料
#如果還沒安裝bs4
# !pip install beautifulsoup4
from bs4 import BeautifulSoup # 這個soup 是用來解析,str 長得很像html 格式的工具 
#如果是mac 再爬取時, 有時候會遇到 ssl 安全憑證問題 , 請幫我加入:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
url = 'https://search.books.com.tw/search/query/key/python/cat/all' #我們的問題(我們的問題,使這個小姐姐給我們一堆盒子)
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'  
}

response = requests.get(url, headers = my_headers)
response.text

In [ ]:
#由於 response.text 不是json 的字典格式, 所以不能用之前的json.loads 的功能
soup = BeautifulSoup(response.text , 'html.parser')
type(soup)
soup

In [ ]:
#開始找盒子, 找盒子功能介紹:
#功能1 find_all -找出很多個我們條件盒子
# 舉例: 嘗試找一下大盒子
boxs = soup.find_all('div',class_ = "table-td") #找到橫向最大的盒子   (品牌, 特徵1 , 特徵2.....)
#由於我們要繼續找盒子(bs4特有的功能), 所以注意一下type要轉換為bs4的type
len(boxs)

# 功能2 find  - 找出一個我們條件的盒子(如果事實上有很多個相同條件盒子,也只會回傳第一個盒子)
box_a   = boxs[19].find('a',target="_blank") 
box_img = box_a.find('img',class_ = 'b-lazy') #b-lazy ,b-loaded 兩種屬性
box_img #找到我們要的img 盒子

#功能3 get  -得到當下盒子裡面其他指定的特徵
img_link= box_img.get('data-src')
title   = box_a.get('title')
print('圖片連結:',img_link)
print('產品title',title)


#功能4 text -得到當下盒子內的紙條
price_box = boxs[19].find('ul',class_ = 'price')
p_list = price_box.find_all('b')
discount = p_list[0].text
price    = p_list[1].text

print(f'產品折扣:{discount}')
print(f'產品價格:{price}')


# 如果想要下載圖片, 使用urllib的功能包
import urllib.request
import os
if not os.path.exists('./imgs'):
    os.makedirs('./imgs')
    
urllib.request.urlretrieve(img_link,'./imgs/1.jpg')
# for each_box in windoes:
#     a_box = each_box.find('a')
#     title= a_box.get('title')
#     link = 'https:'+ a_box.get('href')
#     print(title)
#     print(link)
#     print('=============')



In [ ]:
#整理一下, 圖片連結, 商品名稱 ,價格
print('圖片連結:',img_link)
print('產品title',title)
print('商品價格:',price)


In [2]:
#知道了上面所有功能, 重新爬取第一頁的內容
import time
url = 'https://search.books.com.tw/search/query/key/python/cat/all' #我們的問題(我們的問題,使這個小姐姐給我們一堆盒子)
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'  
}

response = requests.get(url, headers = my_headers)
response.text
soup = BeautifulSoup(response.text , 'html.parser')
boxs = soup.find_all('div',class_ = "table-td")
boxs # 有很多商品box
for idx,box in enumerate(boxs):
    box_a   = box.find('a',target="_blank")
    box_img = box_a.find('img')
    img_link = box_img.get('data-srcset')
    title    = box_a.get('title')
    
    price_box = box.find('ul',class_ = 'price')
    p_list = price_box.find_all('b')[-1]
    price    = p_list.text
    print('圖片連結:',img_link)
    print('商品title:',title)
    print('商品價格:',price)
    print(f'=====================第{idx+1}個box商品爬取完成================================')
    
    
 
    
    
    
    


圖片連結: https://im1.book.com.tw/image/getImage?i=https://addons.books.com.tw/G/prod/comingsoon_rec.gif&w=374&h=374 2x
商品title: Python
商品價格: 1120
=====================第1個box商品爬取完成================================
圖片連結: https://im1.book.com.tw/image/getImage?i=https://addons.books.com.tw/G/prod/comingsoon_rec.gif&w=374&h=374 2x
商品title: Python
商品價格: 1013
=====================第2個box商品爬取完成================================
圖片連結: https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/F01/738/69/F017386990.jpg&w=374&h=374&v=5fa8fffc 2x
商品title: python
商品價格: 1015
=====================第3個box商品爬取完成================================
圖片連結: https://im1.book.com.tw/image/getImage?i=https://addons.books.com.tw/G/prod/comingsoon_rec.gif&w=374&h=374 2x
商品title: Python
商品價格: 688
=====================第4個box商品爬取完成================================
圖片連結: https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/F01/360/97/F013609706.jpg&w=374&h=374&v=5dc31292 2x
商品title: Python
商品價格: 320

In [3]:
# 爬取所有頁面資料
import time 
import random
title_list = []
price_list = []
img_ink_list = []

page_from = 1
while page_from<=5:
    try:
        rst = random.randint(5,10)
        url =  'https://search.books.com.tw/search/query/cat/all/sort/1/v/0/spell/3/ms2/ms2_1/page/'+str(page_from)+'/key/python'
        my_headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'  
        }

        response = requests.get(url, headers = my_headers)
        response.text
        soup = BeautifulSoup(response.text , 'html.parser')
        boxs = soup.find_all('div',class_ = "table-td")
        boxs # 有很多商品box
        print(boxs)
        for idx,box in enumerate(boxs):
            box_a   = box.find('a',target="_blank")
            box_img = box_a.find('img')
            img_link = box_img.get('data-srcset')
            title    = box_a.get('title')

            price_box = box.find('ul',class_ = 'price')
            p_list = price_box.find_all('b')[-1]
            price    = p_list.text
            print('圖片連結:',img_link)
            print('商品title:',title)
            print('商品價格:',price)
    #         print(f'=====================第{idx+1}個box商品爬取完成================================')
            title_list.append(title)
            price_list.append(price)
            img_ink_list.append(img_link)
        print(f'======這頁({page_from})內容爬取完成了,休息{rst}秒========')
        page_from = page_from+1
        
        time.sleep(rst)
    except:
        print('偵測到錯誤')
        time.sleep(5)
        break
    


[]
======這頁(1)內容爬取完成了,休息8秒========
偵測到錯誤


KeyboardInterrupt: 

In [ ]:
#把爬取下來的數據做成表格:
# title_list
# price_list
# img_ink_list
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  

df = pd.DataFrame(columns = ['title','price','img_link'])
df['title']    = title_list
df['price']    = price_list
df['img_link'] = img_ink_list
df
#爬完資料,可以存到我們想要的資料夾
if not os.path.exists('./csv_file'):
    os.makedirs('./csv_file')
df.to_csv('./csv_file/my_file.csv')

In [ ]:
#下載所有圖片
if not os.path.exists('./imgs'):
    os.makedirs('./imgs')
n = 1
for link in df['img_link']:
    link=link.split(' ')
    urllib.request.urlretrieve(link[0],f'./imgs/{n}.jpg')
    n=n+1

In [ ]:
#使用情境: 我只要找有關爬蟲的python書,以及股票爬蟲的書籍
key_word_1 = 'Python'
key_word_2 = '股票'
target_row = []
for row,s in enumerate(df['title']):
    if key_word_1 in s and key_word_2 in s:
        target_row.append(row)
print('與我下的關鍵字有先關的數據位子:',target_row)
target_df = df.iloc[target_row,:]
target_df.to_csv('./csv_file/my_target_file.csv')
target_df

In [ ]:
#把我們感興趣的df price 處理成可以被畫圖的數字型態
def str2int(data):
    return int(data)
    
target_df['price'] = target_df['price'].apply(str2int)
price_list = target_df['price'].values.tolist()
price_list_sorted = sorted(price_list)
price_list_sorted
plt.hist(price_list_sorted,  #hist: 是針對1維數據做記數座圖  #bar:針對2為數據化柱狀圖
        bins=50,
        color = 'green',
        label = 'target_price')
plt.xlabel('price')
plt.ylabel('numbers')
plt.show()


In [ ]:
# 使用需求, 用一句話,搜索整個df 找出最相關的那幾本書
import difflib
def str_similar(s1,s2):
    sim = difflib.SequenceMatcher(None , s1,s2)
    ratio = sim.quick_ratio()
    return ratio 


sentience_key = 'python 資料科學,教你手把手爬取股票分析'
 
ratio_list =[]
for s in df['title']:
    ratio = str_similar(s,sentience_key)
    ratio_list.append(ratio)
ratio_list
df['ratio']=ratio_list
df

In [ ]:
targt_row = []
for row,i in enumerate(df['ratio']):
    if i >0.4:
        target_row.append(row)
target_row
taraget_df = df.iloc[target_row,:]
taraget_df

In [ ]:
#開始包裝上述爬蟲的功能, 包成一個class 方便之後調用
# 爬取所有頁面資料
import time 
import random
import requests
from bs4 import BeautifulSoup
import os
class books_webcrl:
    def __init__(self):
        pass

    def make_dir(self,path): #偷偷註記 path= './img'....
        if not os.path.exists(path):
            os.makedirs(path)


    def webcrl(self,my_key,page_from,target_page):
        title_list = []
        price_list = []
        img_ink_list = []
        while page_from<=target_page:
            try:
                rst = random.randint(10,25)
                url =  'https://search.books.com.tw/search/query/cat/all/sort/1/v/0/spell/3/ms2/ms2_1/page/'+str(page_from)+'/key/'+my_key
                my_headers = {
                    'Host': 'search.books.com.tw',
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.3'  
                }
                response = requests.get(url, headers = my_headers)
                response.text
                soup = BeautifulSoup(response.text , 'html.parser')
                windoes = soup.find_all('table',class_ = "table-searchlist clearfix")
                windows_bs4 = windoes[0]
                boxs = windows_bs4.find_all('tbody')
                boxs # 有很多商品box
                for idx,box in enumerate(boxs):
                    box_a   = box.find('a',target="_blank")
                    box_img = box_a.find('img')
                    img_link = box_img.get('data-srcset')
                    title    = box_a.get('title')

                    boxs_price = box.find_all('td')[2]
                    prices = boxs_price.find('ul', class_="list-nav clearfix")
                    boxs_strong = prices.find_all('strong')
                    price = boxs_strong[-1].text

                    title_list.append(title)
                    price_list.append(price)
                    img_ink_list.append(img_link)
                print(f'======這頁({page_from})內容爬取完成了,休息{rst}秒========')
                page_from = page_from+1

                time.sleep(rst)
            except:
                print('偵測到錯誤')
                time.sleep(5)
                pass
        return title_list , price_list , img_ink_list


In [ ]:
B = books_webcrl()
B.make_dir('./my_imgs')
title_list , price_list , img_ink_list  = B.webcrl('料理',1,3)
#list都有了 開始創建df 做我想要的篩選
df = pd.DataFrame(columns = ['title','price','img_link'])
df['title']    = title_list
df['price']    = price_list
df['img_link'] = img_ink_list
df
#爬完資料,可以存到我們想要的資料夾
if not os.path.exists('./csv_file'):
    os.makedirs('./csv_file')
df.to_csv('./csv_file/my_file.csv')

key_word_1 = '美食'
key_word_2 = '料理'
target_row = []
for row,s in enumerate(df['title']):
    if key_word_1 in s and key_word_2 in s:
        target_row.append(row)
print('與我下的關鍵字有先關的數據位子:',target_row)
target_df = df.iloc[target_row,:]
target_df.to_csv('./csv_file/my_target_file.csv')
target_df